# 1. Recepción de Archivos y Clasificación  
Se implementa una función que recibe un archivo y detecta su tipo.

Librerías utilizadas:
-   os y mimetypes para identificar formatos.
-   pdfplumber y PyMuPDF para PDFs.
-   Pillow para imágenes.
-   pandas y openpyxl para Excel.

In [57]:
import os
import mimetypes

def identificar_tipo_archivo(ruta):
    tipo, _ = mimetypes.guess_type(ruta)
    if tipo:
        tipo_principal, subtipo = tipo.split('/')  # Separa el tipo principal y el subtipo
        if tipo_principal == "image":
            return "imagen"
        elif tipo_principal == "application" and subtipo == "pdf": # Manejo específico para PDF
            return "pdf"
        elif tipo_principal == "application" and "msword" in subtipo or subtipo=="vnd.ms-office":  # Word .doc y .docx
            return "word"
        elif tipo_principal == "application" and "excel" in subtipo or subtipo=="vnd.ms-office": # Excel .xls y .xlsx
            return "excel"
        elif tipo_principal == "text" or subtipo == "csv":  # Texto y CSV
            return "texto"
        elif tipo_principal == "audio":
            return "audio"
        elif tipo_principal == "video":
            return "video"
        elif tipo_principal == "text" and subtipo == "html":
            return "html"
        elif tipo_principal == "text" and subtipo == "css":
            return "css"
        elif tipo_principal == "application" and subtipo == "javascript" or subtipo == "x-javascript":
            return "javascript"
        elif tipo_principal == "application" and subtipo == "json":
            return "json"
        elif tipo_principal == "application" and subtipo == "xml" or subtipo == "atom+xml" or subtipo == "rss+xml": # XML, Atom, RSS
            return "xml"
    return "desconocido"

In [58]:
prueba_pdf = identificar_tipo_archivo('prueba4.pdf')
print(prueba_pdf) 

pdf


# 2. Extracción de Texto según Tipo de Documento

Si es una imagen o PDF escaneado:

- Utilizamos PaddleOCR o EasyOCR.

- Se preprocesa la imagen para mejorar el OCR (ajuste de contraste, umbralización, etc.).

In [59]:
from paddleocr import PaddleOCR
from PIL import Image
import cv2
import numpy as np

def ocr_imagen(ruta):
    ocr = PaddleOCR()
    resultado = ocr.ocr(ruta, cls=True)
    texto_extraido = "\n".join([line[1][0] for line in resultado[0]])
    return texto_extraido

In [60]:
imagen = ocr_imagen('WhatsApp Image 2025-01-20 at 21.04.59.jpeg')
print(imagen)


[2025/02/03 16:02:42] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\OPERADOR/.paddleocr/whl\\det\\ch\\ch_PP-OCRv4_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\OPERADOR/.paddleocr/whl\\rec\\ch\\ch_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batc

Si es un PDF de texto:

- Se extrae con pdfplumber o PyMuPDF.

In [61]:
import pdfplumber
import pandas as pd

# Función para extraer tablas de un PDF y convertirlas en CSV
def pdf_to_csv(pdf_path, output_csv_path):
    with pdfplumber.open(pdf_path) as pdf:
        all_text = ""  # Para guardar el texto extraído
        table_written = False  # Bandera para evitar duplicar encabezados en CSV

        for page_num, page in enumerate(pdf.pages):
            # Extraer texto antes de las tablas
            page_text = page.extract_text()
            all_text += page_text  # Guardar el texto de cada página

            # Extraer tablas
            table = page.extract_table()

            if table:
                # Convertir la tabla en un DataFrame y agregar al CSV
                df = pd.DataFrame(table[1:], columns=table[0])  # Excluir la primera fila de encabezado
                if not table_written:
                    # Escribir el encabezado solo en la primera tabla
                    df.to_csv(output_csv_path, mode='w', header=True, index=False)
                    table_written = True
                else:
                    # Si ya se ha escrito una tabla, agregar solo los datos
                    df.to_csv(output_csv_path, mode='a', header=False, index=False)

    # Si también quieres guardar el texto completo de las páginas en un archivo
    # guardarlo en un archivo de texto, por ejemplo:
    text_output_path = "extracted_text.txt"
    with open(text_output_path, "w", encoding="utf-8") as text_file:
        text_file.write(all_text)
        
    print("Extracción completada. El texto y las tablas se han guardado correctamente.")

# Ejemplo de uso:
pdf_to_csv("PRUEBA2.pdf", "output2.csv")


Extracción completada. El texto y las tablas se han guardado correctamente.


In [62]:
# Llamar a la función
pdf_to_csv("PRUEBA2.pdf", "salida.csv")

Extracción completada. El texto y las tablas se han guardado correctamente.


Si es un archivo de texto o Excel:

- pandas y openpyxl permiten extraer los datos directamente.

In [63]:
import pandas as pd

def leer_excel(ruta):
    df = pd.read_excel(ruta)
    return df.to_string()

# 3. Procesamiento de Texto con REDES NEURONALES (NLP y Normalización)

- Utilizamos BERT, Spacy y Regex para estructurar los datos.

- Se identifican palabras clave como nombres de productos, códigos y cantidades.


In [64]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image
import pandas as pd
import io

# Función para extraer imágenes de un PDF
def extract_images_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    images = []

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        img_list = page.get_images(full=True)  # Extrae imágenes embebidas
        
        for img_index, img in enumerate(img_list):
            xref = img[0]
            base_image = pdf_document.extract_image(xref)  # Extrae la imagen
            img_bytes = base_image["image"]  # Bytes de la imagen
            img_pil = Image.open(io.BytesIO(img_bytes))  # Convertir a PIL Image
            images.append(img_pil)

    return images

# Función para realizar OCR en las imágenes extraídas
def extract_text_from_images(images):
    full_text = []

    for img in images:
        text = pytesseract.image_to_string(img, lang="eng+spa")  # OCR en inglés y español
        full_text.append(text)

    return full_text

# Función para guardar el texto extraído en un CSV
def save_text_to_csv(text_list, output_csv):
    df = pd.DataFrame({"Extracted_Text": text_list})
    df.to_csv(output_csv, index=False, encoding="utf-8")

# Función principal: convierte PDF con imágenes a CSV
def pdf_to_csv(pdf_path, output_csv="output.csv"):
    images = extract_images_from_pdf(pdf_path)  # Extraer imágenes del PDF
    extracted_text = extract_text_from_images(images)  # Hacer OCR en las imágenes
    save_text_to_csv(extracted_text, output_csv)  # Guardar en CSV

# Ejemplo de uso:
pdf_to_csv("Captura de pantalla 2025-02-03 153436.pdf", "PRUEBA.csv")
print("Conversión completada. El texto extraído se ha guardado en 'salida.csv'.")

Conversión completada. El texto extraído se ha guardado en 'salida.csv'.


In [65]:
import fitz  # PyMuPDF
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
import pandas as pd
import io

# Función para mejorar la imagen (contraste y nitidez)
def preprocess_image(image):
    # Convertir a escala de grises
    image = image.convert("L")  

    # Aumentar contraste
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(3)  # Ajusta el contraste (valor 3 es bastante fuerte)

    # Filtrar ruido y mejorar nitidez
    image = image.filter(ImageFilter.SHARPEN)

    return image

# Función para extraer imágenes de un PDF
def extract_images_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    images = []

    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        img_list = page.get_images(full=True)  

        for img in img_list:
            xref = img[0]
            base_image = pdf_document.extract_image(xref)
            img_bytes = base_image["image"]
            img_pil = Image.open(io.BytesIO(img_bytes))

            # Preprocesar la imagen para mejorar OCR
            img_pil = preprocess_image(img_pil)
            images.append(img_pil)

    return images

# Función para realizar OCR en imágenes mejoradas
def extract_text_from_images(images):
    full_text = []

    for img in images:
        text = pytesseract.image_to_string(img, lang="eng+spa")  # OCR en inglés y español
        full_text.append(text)

    return full_text

# Función para guardar el texto extraído en un CSV
def save_text_to_csv(text_list, output_csv):
    df = pd.DataFrame({"Extracted_Text": text_list})
    df.to_csv(output_csv, index=False, encoding="utf-8")

# Función principal
def pdf_to_csv(pdf_path, output_csv="output.csv"):
    images = extract_images_from_pdf(pdf_path)  # Extraer imágenes
    extracted_text = extract_text_from_images(images)  # OCR en imágenes procesadas
    save_text_to_csv(extracted_text, output_csv)  # Guardar en CSV

# Ejemplo de uso:
pdf_to_csv("PRUEBA1.pdf", "PRUEBA1.csv")
print("Conversión completada. El texto extraído se ha guardado en 'salida.csv'.")


Conversión completada. El texto extraído se ha guardado en 'salida.csv'.


# 4. Conversión del Texto Extraído a CSV
- Se estructura en un DataFrame y se guarda en CSV o Excel.

In [66]:
def guardar_csv(datos, nombre_archivo="output.csv"):
    df = pd.DataFrame(datos)
    df.to_csv(nombre_archivo, index=False, encoding="utf-8-sig")

In [67]:
#df_normalizar_texto  = guardar_csv(pdf_normalizar_texto)